# House Keeping

In [1]:
from fastai.conv_learner import *
from fastai.dataset import *

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
PATH = './'
TRAIN = 'data/train/'
TEST = 'data/test/'
LABELS = 'data/train.csv'
SAMPLE = 'data/sample_submission.csv'

In [3]:
train_names = list({f[:36] for f in os.listdir(TRAIN)})
test_names = list({f[:36] for f in os.listdir(TEST)})
tr_n, val_n = train_test_split(train_names, test_size=0.1, random_state=42)

In [4]:
def open_rgby(path,id): 
    colors = ['red','green','blue','yellow']
    flags = cv2.IMREAD_GRAYSCALE
    img = [cv2.imread(os.path.join(path, id+'_'+color+'.png'), flags).astype(np.float32)/255
           for color in colors]
    return np.stack(img, axis=-1)

In [5]:
name_label_dict = {
0:  'Nucleoplasm',
1:  'Nuclear membrane',
2:  'Nucleoli',   
3:  'Nucleoli fibrillar center',
4:  'Nuclear speckles',
5:  'Nuclear bodies',
6:  'Endoplasmic reticulum',   
7:  'Golgi apparatus',
8:  'Peroxisomes',
9:  'Endosomes',
10:  'Lysosomes',
11:  'Intermediate filaments',
12:  'Actin filaments',
13:  'Focal adhesion sites',   
14:  'Microtubules',
15:  'Microtubule ends',  
16:  'Cytokinetic bridge',   
17:  'Mitotic spindle',
18:  'Microtubule organizing center',  
19:  'Centrosome',
20:  'Lipid droplets',
21:  'Plasma membrane',   
22:  'Cell junctions', 
23:  'Mitochondria',
24:  'Aggresome',
25:  'Cytosol',
26:  'Cytoplasmic bodies',   
27:  'Rods & rings' }

# Data Objects

In [6]:
class pdFilesDataset(FilesDataset):
    def __init__(self, fnames, path, transform):
        self.labels = pd.read_csv(LABELS).set_index('Id')
        self.labels['Target'] = [[int(i) for i in s.split()] for s in self.labels['Target']]
        super().__init__(fnames, transform, path)
    
    def get_x(self, i):
        img = open_rgby(self.path,self.fnames[i])
        if self.sz == 512: return img 
        else: return cv2.resize(img, (self.sz, self.sz),cv2.INTER_AREA)
    
    def get_y(self, i):
        if(self.path == TEST): return np.zeros(len(name_label_dict),dtype=np.int)
        else:
            labels = self.labels.loc[self.fnames[i]]['Target']
            return np.eye(len(name_label_dict),dtype=np.float)[labels].sum(axis=0)
        
    @property
    def is_multi(self): return True
    @property
    def is_reg(self):return True
    #this flag is set to remove the output sigmoid that allows log(sigmoid) optimization
    #of the numerical stability of the loss function
    
    def get_c(self): return len(name_label_dict) #number of classes

In [7]:
def get_data(sz,bs):
    #data augmentation
    aug_tfms = [RandomRotate(30, tfm_y=TfmType.NO),
                RandomDihedral(tfm_y=TfmType.NO)]
    stats = A([0.00505, 0.00331, 0.00344, 0.00519], [0.10038, 0.08131, 0.08284, 0.10179])
    tfms = tfms_from_stats(stats, sz, crop_type=CropType.NO, tfm_y=TfmType.NO, 
                aug_tfms=aug_tfms)
    ds = ImageData.get_ds(pdFilesDataset, (tr_n[:-(len(tr_n)%bs)],TRAIN), 
                (val_n,TRAIN), tfms, test=(test_names,TEST))
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
    return md

### Stratified Folds

In [24]:
from sklearn.model_selection import StratifiedKFold

folds = 5
train_folds = []
val_folds = []

def make_folds():
    skf = StratifiedKFold(n_splits=folds, random_state=33, shuffle=True)
    
    train_names = list({f[:36] for f in os.listdir(TRAIN)})
    test_names = list({f[:36] for f in os.listdir(TEST)})
    trn_df = pd.read_csv('data/train.csv')
    
    for train_index, evaluate_index in skf.split(trn_df.index.values, trn_df.Target):
        trn_value = trn_df.iloc[train_index]
        val_value = trn_df.iloc[evaluate_index]
        train_folds.append(trn_value)
        val_folds.append(val_value)
        print(train_index.shape, evaluate_index.shape)
    
    for i in range(folds):
        train_folds[i].to_csv(f'data/5_fold/trn_folds_{i}')
        val_folds[i].to_csv(f'data/5_fold/val_folds_{i}')

def load_folds():
    for i in range(folds):
        train_folds.append(pd.read_csv(f'data/5_fold/trn_folds_{i}'))
        val_folds.append(pd.read_csv(f'data/5_fold/val_folds_{i}'))
        print(len(train_folds[i]), len(val_folds[i]))

In [25]:
# make_folds()
load_folds()

24673 6399
24836 6236
24710 6362
24997 6075
25072 6000


### Over sampling

In [194]:
from sklearn.model_selection import train_test_split

folds = 5
over_train_folds = []
over_val_folds = []

def make_over_folds():
    skf = StratifiedKFold(n_splits=folds, random_state=33, shuffle=True)
    
    trn_df = pd.read_csv('data/train.csv')
    
    for train_index, evaluate_index in skf.split(trn_df.index.values, trn_df.Target):
        trn_value = trn_df.iloc[train_index]
        val_value = trn_df.iloc[evaluate_index]
        
        over_train_folds.append(trn_value)
        over_val_folds.append(val_value)
        print(train_index.shape, evaluate_index.shape)
    
    for k in range(folds):
        trn_fold_frame = over_train_folds[k].copy()
        val_fold_frame = over_val_folds[k].copy()
        
        trn_unique = trn_fold_frame[~trn_fold_frame['Target'].duplicated(keep=False)]
        val_unique = val_fold_frame[~val_fold_frame['Target'].duplicated(keep=False)]
        
        trn_unique = pd.concat([trn_unique]*100, ignore_index=True)
        val_unique = pd.concat([val_unique]*20, ignore_index=True)
        
        over_train_folds[k] = pd.concat([trn_fold_frame, trn_unique], ignore_index=True)
        over_val_folds[k] = pd.concat([val_fold_frame, val_unique], ignore_index=True)
        
    for i in range(folds):
        over_train_folds[i].to_csv(f'data/5_fold/over_sampled/trn_folds_{i}')
        over_val_folds[i].to_csv(f'data/5_fold/over_sampled/val_folds_{i}')

def load_over_folds():
    for i in range(folds):
        over_train_folds.append(pd.read_csv(f'data/5_fold/over_sampled/trn_folds_{i}'))
        over_val_folds.append(pd.read_csv(f'data/5_fold/over_sampled/val_folds_{i}'))

In [195]:
# make_over_folds()

/home/eigenstir/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


(24673,) (6399,)
(24836,) (6236,)
(24710,) (6362,)
(24997,) (6075,)
(25072,) (6000,)


In [ ]:
load_over_folds()

In [ ]:
for i in range(folds):
    print(len(over_train_folds[i]))
    print(len(over_val_folds[i]))

# Loss Functions

In [12]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()

        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.sum(dim=1).mean()
    
def acc(preds,targs,thresh=0.0):
    preds = (preds > thresh).int()
    targs = targs.int()
    return (preds==targs).float().mean()

def f1_loss(y_true, y_pred):
    epsilon = 1e-7
    y_pred = torch.sigmoid(y_pred)
    y_true = y_true.type(torch.FloatTensor).cuda()
    tp = torch.sum(y_true*y_pred)
    tn = torch.sum((1-y_true)*(1-y_pred))
    fp = torch.sum((1-y_true)*y_pred)
    fn = torch.sum((y_true*(1-y_pred)))
    
    p = tp / (tp + fp + epsilon)
    r = tp / (tp + fn + epsilon)
    f1 = 2*p*r / (p+r+epsilon)
    f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1), f1)
    return 2 - torch.mean(f1)

def f1_metric(y_true, y_pred):
    y_pred = torch.sigmoid(y_pred)
    y_true = y_true.type(torch.FloatTensor).cuda()
    score = torch.sum(2.0*(y_pred*y_true))/torch.sum((y_pred+y_true) + 1e-7)
    return score

# Model

## Conv Base

In [13]:
class ConvnetBuilder_custom():
    def __init__(self, f, c, is_multi, is_reg, ps=None, xtra_fc=None, xtra_cut=0, 
                 custom_head=None, pretrained=True):
        self.f,self.c,self.is_multi,self.is_reg,self.xtra_cut = f,c,is_multi,is_reg,xtra_cut
        if xtra_fc is None: xtra_fc = [512]
        if ps is None: ps = [0.25]*len(xtra_fc) + [0.5]
        self.ps,self.xtra_fc = ps,xtra_fc

        if f in model_meta: cut,self.lr_cut = model_meta[f]
        else: cut,self.lr_cut = 0,0
        cut-=xtra_cut
        layers = cut_model(f(pretrained), cut)
        
        #replace first convolutional layer by 4->64 while keeping corresponding weights
        #and initializing new weights with zeros
        #####################################################
        w = layers[0].weight
        layers[0] = nn.Conv2d(4,64,kernel_size=(7,7),stride=(2,2),padding=(3, 3), bias=False)
        layers[0].weight = torch.nn.Parameter(torch.cat((w,torch.zeros(64,1,7,7)),dim=1))
        #####################################################
        
        self.nf = model_features[f] if f in model_features else (num_features(layers)*2)
        if not custom_head: layers += [AdaptiveConcatPool2d(), Flatten()]
        self.top_model = nn.Sequential(*layers)

        n_fc = len(self.xtra_fc)+1
        if not isinstance(self.ps, list): self.ps = [self.ps]*n_fc

        if custom_head: fc_layers = [custom_head]
        else: fc_layers = self.get_fc_layers()
        self.n_fc = len(fc_layers)
        self.fc_model = to_gpu(nn.Sequential(*fc_layers))
        if not custom_head: apply_init(self.fc_model, kaiming_normal)
        self.model = to_gpu(nn.Sequential(*(layers+fc_layers)))

    @property
    def name(self): return f'{self.f.__name__}_{self.xtra_cut}'

    def create_fc_layer(self, ni, nf, p, actn=None):
        res=[nn.BatchNorm1d(num_features=ni)]
        if p: res.append(nn.Dropout(p=p))
        res.append(nn.Linear(in_features=ni, out_features=nf))
        if actn: res.append(actn)
        return res

    def get_fc_layers(self):
        res=[]
        ni=self.nf
        for i,nf in enumerate(self.xtra_fc):
            res += self.create_fc_layer(ni, nf, p=self.ps[i], actn=nn.ReLU())
            ni=nf
        final_actn = nn.Sigmoid() if self.is_multi else nn.LogSoftmax()
        if self.is_reg: final_actn = None
        res += self.create_fc_layer(ni, self.c, p=self.ps[-1], actn=final_actn)
        return res

    def get_layer_groups(self, do_fc=False):
        if do_fc:
            return [self.fc_model]
        idxs = [self.lr_cut]
        c = children(self.top_model)
        if len(c)==3: c = children(c[0])+c[1:]
        lgs = list(split_by_idxs(c,idxs))
        return lgs+[self.fc_model]
    
class ConvLearner(Learner):
    def __init__(self, data, models, precompute=False, **kwargs):
        self.precompute = False
        super().__init__(data, models, **kwargs)
        if hasattr(data, 'is_multi') and not data.is_reg and self.metrics is None:
            self.metrics = [accuracy_thresh(0.5)] if self.data.is_multi else [accuracy]
        if precompute: self.save_fc1()
        self.freeze()
        self.precompute = precompute

    def _get_crit(self, data):
        if not hasattr(data, 'is_multi'): return super()._get_crit(data)

        return F.l1_loss if data.is_reg else F.binary_cross_entropy if data.is_multi else F.nll_loss

    @classmethod
    def pretrained(cls, f, data, ps=None, xtra_fc=None, xtra_cut=0, custom_head=None, precompute=False,
                   pretrained=True, **kwargs):
        models = ConvnetBuilder_custom(f, data.c, data.is_multi, data.is_reg,
            ps=ps, xtra_fc=xtra_fc, xtra_cut=xtra_cut, custom_head=custom_head, pretrained=pretrained)
        return cls(data, models, precompute, **kwargs)

    @classmethod
    def lsuv_learner(cls, f, data, ps=None, xtra_fc=None, xtra_cut=0, custom_head=None, precompute=False,
                  needed_std=1.0, std_tol=0.1, max_attempts=10, do_orthonorm=False, **kwargs):
        models = ConvnetBuilder(f, data.c, data.is_multi, data.is_reg,
            ps=ps, xtra_fc=xtra_fc, xtra_cut=xtra_cut, custom_head=custom_head, pretrained=False)
        convlearn=cls(data, models, precompute, **kwargs)
        convlearn.lsuv_init()
        return convlearn
    
    @property
    def model(self): return self.models.fc_model if self.precompute else self.models.model
    
    def half(self):
        if self.fp16: return
        self.fp16 = True
        if type(self.model) != FP16: self.models.model = FP16(self.model)
        if not isinstance(self.models.fc_model, FP16): self.models.fc_model = FP16(self.models.fc_model)
    def float(self):
        if not self.fp16: return
        self.fp16 = False
        if type(self.models.model) == FP16: self.models.model = self.model.module.float()
        if type(self.models.fc_model) == FP16: self.models.fc_model = self.models.fc_model.module.float()

    @property
    def data(self): return self.fc_data if self.precompute else self.data_

    def create_empty_bcolz(self, n, name):
        return bcolz.carray(np.zeros((0,n), np.float32), chunklen=1, mode='w', rootdir=name)

    def set_data(self, data, precompute=False):
        super().set_data(data)
        if precompute:
            self.unfreeze()
            self.save_fc1()
            self.freeze()
            self.precompute = True
        else:
            self.freeze()

    def get_layer_groups(self):
        return self.models.get_layer_groups(self.precompute)

    def summary(self):
        precompute = self.precompute
        self.precompute = False
        res = super().summary()
        self.precompute = precompute
        return res

    def get_activations(self, force=False):
        tmpl = f'_{self.models.name}_{self.data.sz}.bc'
        # TODO: Somehow check that directory names haven't changed (e.g. added test set)
        names = [os.path.join(self.tmp_path, p+tmpl) for p in ('x_act', 'x_act_val', 'x_act_test')]
        if os.path.exists(names[0]) and not force:
            self.activations = [bcolz.open(p) for p in names]
        else:
            self.activations = [self.create_empty_bcolz(self.models.nf,n) for n in names]

    def save_fc1(self):
        self.get_activations()
        act, val_act, test_act = self.activations
        m=self.models.top_model
        if len(self.activations[0])!=len(self.data.trn_ds):
            predict_to_bcolz(m, self.data.fix_dl, act)
        if len(self.activations[1])!=len(self.data.val_ds):
            predict_to_bcolz(m, self.data.val_dl, val_act)
        if self.data.test_dl and (len(self.activations[2])!=len(self.data.test_ds)):
            if self.data.test_dl: predict_to_bcolz(m, self.data.test_dl, test_act)

        self.fc_data = ImageClassifierData.from_arrays(self.data.path,
                (act, self.data.trn_y), (val_act, self.data.val_y), self.data.bs, classes=self.data.classes,
                test = test_act if self.data.test_dl else None, num_workers=8)

    def freeze(self):
        self.freeze_to(-1)

    def unfreeze(self):
        self.freeze_to(0)
        self.precompute = False

    def predict_array(self, arr):
        precompute = self.precompute
        self.precompute = False
        pred = super().predict_array(arr)
        self.precompute = precompute
        return pred

## SE Definitions

In [14]:
pretrained_settings = {
    'se_resnext50_32x4d': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
    'se_resnext101_32x4d': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
}

class SEModule(nn.Module):
    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1,
                             padding=0)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1,
                             padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return module_input * x


class Bottleneck(nn.Module):
    """
    Base class for bottlenecks that implements `forward()` method.
    """
    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.drop1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.drop2(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = self.se_module(out) + residual
        out = self.relu(out)

        return out


class SEBottleneck(Bottleneck):
    """
    Bottleneck for SENet154.
    """
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None):
        super(SEBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes * 2, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes * 2)
        self.conv2 = nn.Conv2d(planes * 2, planes * 4, kernel_size=3,
                               stride=stride, padding=1, groups=groups,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(planes * 4)
        self.conv3 = nn.Conv2d(planes * 4, planes * 4, kernel_size=1,
                               bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride


class SEResNetBottleneck(Bottleneck):
    """
    ResNet bottleneck with a Squeeze-and-Excitation module. It follows Caffe
    implementation and uses `stride=stride` in `conv1` and not in `conv2`
    (the latter is used in the torchvision implementation of ResNet).
    """
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None):
        super(SEResNetBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False,
                               stride=stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1,
                               groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride


class SEResNeXtBottleneck(Bottleneck):
    """
    ResNeXt bottleneck type C with a Squeeze-and-Excitation module.
    """
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None, base_width=4):
        super(SEResNeXtBottleneck, self).__init__()
        width = math.floor(planes * (base_width / 64)) * groups
        self.conv1 = nn.Conv2d(inplanes, width, kernel_size=1, bias=False,
                               stride=1)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride,
                               padding=1, groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = nn.Conv2d(width, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride
        
        self.drop1 = nn.Dropout2d(0.2)
        self.drop2 = nn.Dropout2d(0.2)

class SENet(nn.Module):
    def __init__(self, block, layers, groups, reduction, dropout_p=0.5,
                 inplanes=128, input_3x3=True, downsample_kernel_size=3,
                 downsample_padding=1, num_classes=1000):
        super(SENet, self).__init__()
        self.inplanes = inplanes
        if input_3x3:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, 64, 3, stride=2, padding=1,
                                    bias=False)),
                ('bn1', nn.BatchNorm2d(64)),
                ('relu1', nn.ReLU(inplace=True)),
                ('conv2', nn.Conv2d(64, 64, 3, stride=1, padding=1,
                                    bias=False)),
                ('bn2', nn.BatchNorm2d(64)),
                ('relu2', nn.ReLU(inplace=True)),
                ('conv3', nn.Conv2d(64, inplanes, 3, stride=1, padding=1,
                                    bias=False)),
                ('bn3', nn.BatchNorm2d(inplanes)),
                ('relu3', nn.ReLU(inplace=True)),
            ]
        else:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, inplanes, kernel_size=7, stride=2,
                                    padding=3, bias=False)),
                ('bn1', nn.BatchNorm2d(inplanes)),
                ('relu1', nn.ReLU(inplace=True)),
            ]
        # To preserve compatibility with Caffe weights `ceil_mode=True`
        # is used instead of `padding=1`.
        layer0_modules.append(('pool', nn.MaxPool2d(3, stride=2,
                                                    ceil_mode=True)))
        self.layer0 = nn.Sequential(OrderedDict(layer0_modules))
        self.layer1 = self._make_layer(
            block,
            planes=64,
            blocks=layers[0],
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=1,
            downsample_padding=0
        )
        self.layer2 = self._make_layer(
            block,
            planes=128,
            blocks=layers[1],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.layer3 = self._make_layer(
            block,
            planes=256,
            blocks=layers[2],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.layer4 = self._make_layer(
            block,
            planes=512,
            blocks=layers[3],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropout_p) if dropout_p is not None else None
        self.last_linear = nn.Linear(512 * block.expansion, num_classes)
#         self.last_linear = nn.Linear(512*16, num_classes)

    def _make_layer(self, block, planes, blocks, groups, reduction, stride=1,
                    downsample_kernel_size=1, downsample_padding=0):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=downsample_kernel_size, stride=stride,
                          padding=downsample_padding, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, groups, reduction, stride,
                            downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups, reduction))

        return nn.Sequential(*layers)

    def features(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def logits(self, x):
        x = self.avg_pool(x)
        if self.dropout is not None:
            x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, x):
        x = self.features(x)
        x = self.logits(x)
        return x


def initialize_pretrained_model(model, num_classes, settings):
    assert num_classes == settings['num_classes'], \
        'num_classes should be {}, but is {}'.format(
            settings['num_classes'], num_classes)
    model.load_state_dict(model_zoo.load_url(settings['url']))
    model.input_space = settings['input_space']
    model.input_size = settings['input_size']
    model.input_range = settings['input_range']
    model.mean = settings['mean']
    model.std = settings['std']

In [227]:
class ResNextConvBuilder(ConvnetBuilder_custom):
    def __init__(self, f, c, is_multi, is_reg, ps=None, xtra_fc=None, xtra_cut=0, 
                 custom_head=None, pretrained=True):
        self.f,self.c,self.is_multi,self.is_reg,self.xtra_cut = f,c,is_multi,is_reg,xtra_cut
        if xtra_fc is None: xtra_fc = [512]
        if ps is None: ps = [0.25]*len(xtra_fc) + [0.5]
        self.ps,self.xtra_fc = ps,xtra_fc

        if f in model_meta: cut,self.lr_cut = model_meta[f]
        else: cut,self.lr_cut = 8,6
        cut-=xtra_cut
        layers = cut_model(f(), 5)
        
        #replace first convolutional layer by 4->64 while keeping corresponding weights
        #and initializing new weights with zeros
        #####################################################
        w = layers[0].conv1.weight
        w1 = layers[0].conv1.weight[:,0].unsqueeze(dim=1)
        layers[0].conv1 = nn.Conv2d(4,64,kernel_size=(7,7),stride=(2,2),padding=(3, 3), bias=False)
        layers[0].conv1.weight = torch.nn.Parameter(torch.cat((w,w1),dim=1))
        #####################################################
        
        self.nf = model_features[f] if f in model_features else (num_features(layers)*2)
        if not custom_head: layers += [AdaptiveConcatPool2d(), Flatten()]
        self.top_model = nn.Sequential(*layers)

        n_fc = len(self.xtra_fc)+1
        if not isinstance(self.ps, list): self.ps = [self.ps]*n_fc

        if custom_head: fc_layers = [custom_head]
        else: fc_layers = self.get_fc_layers()
        self.n_fc = len(fc_layers)
        self.fc_model = to_gpu(nn.Sequential(*fc_layers))
        if not custom_head: apply_init(self.fc_model, kaiming_normal)
        self.model = to_gpu(nn.Sequential(*(layers+fc_layers)))

class ResNextConv(ConvLearner):
        @classmethod
        def pretrained(cls, f, data, ps=None, xtra_fc=None, xtra_cut=0, custom_head=None, precompute=False,
                   pretrained=True, **kwargs):
            models = ResNextConvBuilder(f, data.c, data.is_multi, data.is_reg,
            ps=ps, xtra_fc=xtra_fc, xtra_cut=xtra_cut, custom_head=custom_head, pretrained=pretrained)
            return cls(data, models, precompute, **kwargs)

## Model Functions

In [228]:
def se_resnext50_32x4d(num_classes=1000, pretrained='imagenet'):
    model = SENet(SEResNeXtBottleneck, [3, 4, 6, 3], groups=32, reduction=16,
                  dropout_p=0.5, inplanes=64, input_3x3=False,
                  downsample_kernel_size=1, downsample_padding=0,
                  num_classes=num_classes)
    if pretrained is not None:
        settings = pretrained_settings['se_resnext50_32x4d'][pretrained]
        initialize_pretrained_model(model, num_classes, settings)
    return model

In [229]:
def get_fold(sz, bs, k):
    aug_tfms = [RandomRotate(30, tfm_y=TfmType.NO),
                RandomDihedral(tfm_y=TfmType.NO),
                RandomLighting(0.05, 0.05, tfm_y=TfmType.NO)]
    
    stats = A([0.00505, 0.00331, 0.00344, 0.00519], [0.10038, 0.08131, 0.08284, 0.10179])
    tfms = tfms_from_stats(stats, sz, crop_type=CropType.NO, tfm_y=TfmType.NO, 
                aug_tfms=aug_tfms)
    
    trn_x = list(train_folds[k]['Id'])
    val_x = list(val_folds[k]['Id'])
    
    if len(trn_x)%bs == 0:
        ds = ImageData.get_ds(pdFilesDataset, (trn_x,TRAIN), 
                (val_x,TRAIN), tfms, test=(test_names,TEST))
    else:
        ds = ImageData.get_ds(pdFilesDataset, (trn_x[:-(len(trn_x)%bs)],TRAIN), 
                (val_x,TRAIN), tfms, test=(test_names,TEST))
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
    return md

In [230]:
def get_resnext50_model(md):
    learn = ResNextConv.pretrained(se_resnext50_32x4d, md, ps=0.5) #dropout 50%
    learn.opt_fn = optim.Adam
    learn.crit = FocalLoss()
    learn.metrics = [acc, f1_metric]
    learn.clip = 1.0
    return learn 

In [231]:
def get_fold_model(sz, bs, k):
    md = get_fold(sz,bs,k)
    learn = get_resnext50_model(md)
    learn.freeze_to(1)
    return learn

### Oversample Edition

In [232]:
def get_over_fold(sz, bs, k):
    aug_tfms = [RandomRotate(30, tfm_y=TfmType.NO),
                RandomDihedral(tfm_y=TfmType.NO),
                RandomLighting(0.05, 0.05, tfm_y=TfmType.NO)]
    
    stats = A([0.00505, 0.00331, 0.00344, 0.00519], [0.10038, 0.08131, 0.08284, 0.10179])
    tfms = tfms_from_stats(stats, sz, crop_type=CropType.NO, tfm_y=TfmType.NO, 
                aug_tfms=aug_tfms)
    
    trn_x = list(over_train_folds[k]['Id'])
    val_x = list(over_val_folds[k]['Id'])
    
    if len(trn_x)%bs == 0:
        ds = ImageData.get_ds(pdFilesDataset, (trn_x,TRAIN), 
                (val_x,TRAIN), tfms, test=(test_names,TEST))
    else:
        ds = ImageData.get_ds(pdFilesDataset, (trn_x[:-(len(trn_x)%bs)],TRAIN), 
                (val_x,TRAIN), tfms, test=(test_names,TEST))
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
    return md

def get_overfold_model(sz, bs, k):
    md = get_over_fold(sz,bs,k)
    learn = get_resnext50_model(md)
    learn.freeze_to(1)
    return learn

# Train model

## Overfold Experiments

In [216]:
lr = 3e-4
wd = 1e-7
nw=6
lrs=np.array([lr/10,lr/3,lr])

In [200]:
sz = 128
bs = 24

learn = get_overfold_model(sz,bs,3)
learn.unfreeze()
learn.fit(lrs, 1, wds=wd, cycle_len=10, use_clr_beta=(10,10,0.85,0.9), use_wd_sched=True)
learn.save('SEResNextFold_128_3')

/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


epoch      trn_loss   val_loss   acc        f1_metric         
    0      2.894141   1.894364   0.928073   1.894287  
    1      1.952773   1.655329   0.931392   1.596589          
    2      1.72733    1.555247   0.934232   1.658985          
    3      1.606527   1.537318   0.934829   1.558331          
    4      1.499816   1.553119   0.935476   1.48484           
    5      1.39727    1.567557   0.933891   1.442354          
    6      1.312646   1.592852   0.932178   1.413704          
    7      1.277144   1.582182   0.931655   1.410893          
    8      1.274289   1.574093   0.932425   1.40843           
    9      1.240694   1.591768   0.932079   1.400213          



In [203]:
sz = 256
bs = 16

learn = get_overfold_model(sz,bs,3)
learn.load('SEResNextFold_128_3')
learn.unfreeze()
learn.fit(lrs, 1, wds=wd, cycle_len=10, use_clr_beta=(20,8,0.85,0.9), use_wd_sched=True)
learn.save('SEResNextFold_256_3')

/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


epoch      trn_loss   val_loss   acc        f1_metric         
    0      1.287492   1.480285   0.933829   1.411398  
    1      1.190252   1.432211   0.936756   1.385831          
    2      1.07842    1.427014   0.938588   1.354518          
    3      1.050972   1.409981   0.939156   1.326916          
    4      0.941893   1.369003   0.942046   1.312287           
    5      0.877585   1.367926   0.942392   1.289273           
    6      0.863571   1.329484   0.944335   1.286037           
    7      0.830794   1.270952   0.94522    1.283957           
    8      0.795969   1.318031   0.945266   1.267977           
    9      0.800551   1.284401   0.945784   1.275932           


In [236]:
sz = 512
bs = 8

learn = get_overfold_model(sz,bs,3)
learn.load('SEResNextFold_256_3')
learn.unfreeze()
learn.fit(lrs, 1, wds=wd, cycle_len=20, use_clr_beta=(20,8,0.85,0.9), use_wd_sched=True, best_save_name='SEResNextFold_512_3best')
learn.save('SEResNextFold_512_3')

/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


epoch      trn_loss   val_loss   acc        f1_metric          
    0      0.936123   1.493907   0.939811   1.257735  
    1      0.909803   1.498809   0.942874   1.244564           
    2      0.909558   1.519219   0.943788   1.222911           
    3      0.874937   1.512659   0.94534    1.212539           
    4      0.770423   1.367456   0.948469   1.219535           
    5      0.801867   1.5168     0.947616   1.199073           
    6      0.761069   1.317298   0.951853   1.197406           
    7      0.714581   1.306861   0.951408   1.199208           
    8      0.699356   1.419535   0.953409   1.186518           
    9      0.662387   1.282365   0.955352   1.191486           
    10     0.656751   1.251562   0.953623   1.178844           
    11     0.659572   1.137671   0.956575   1.189399           
    12     0.6996     1.205377   0.957271   1.179339           
    13     0.662278   1.083555   0.958654   1.178905           
    14     0.626937   1.007507   0.959074   1.192

In [237]:
pred = make_prediction()
save_pred(pred, th_t, fname=f'protein_class_3_oversample_tht.csv')
save_pred(pred, 0.5, fname=f'protein_class_3_oversample_0.5.csv')

# Cosine Annealing

In [ ]:
def get_resnext50_model(md):
    learn = ResNextConv.pretrained(se_resnext50_32x4d, md, ps=0.5) #dropout 50%
    learn.opt_fn = optim.Adam
    learn.crit = FocalLoss()
    learn.metrics = [acc, f1_metric]
    learn.clip = 0.1 
    return learn 

In [ ]:
def get_fold_model(sz, bs, k):
    md = get_fold(sz,bs,k)
    learn = get_resnext50_model(md)
    return learn

In [ ]:
lr = 3e-4
wd = 1e-7
nw = 6
lrs=np.array([lr/10,lr/3,lr])

for k in range(folds):
    # 128x128 phase
    learn = get_fold_model(128, 24, k)
    
    learn.fit(lr,1,wds=wd,cycle_len=8,use_clr=(5,8))

    learn.unfreeze()
    learn.bn_freeze(True)

    learn.fit(lrs/2, 1, wds=wd, cycle_len=10,use_clr=(20,10))

    learn.save(f'128_SeResNext50_{k}')

    # 256 x 256 phase
    learn = get_fold_model(256, 16, k)
    learn.load(f'128_SeResNext50_{k}')

    learn.fit(lr,1,wds=wd, cycle_len=5,use_clr=(5,5))

    learn.unfreeze()
    learn.bn_freeze(True)

    learn.fit(lrs/2,2,wds=wd, cycle_len=10,use_clr=(20,8))

    learn.save(f'256_SeResNext50_{k}')

    #512 x 512 phase
    learn = get_fold_model(512, 8, k)
    learn.load(f'256_SeResNext50_{k}')

    learn.fit(lr,1, wds=wd, cycle_len=2,use_clr=(5,4))

    learn.unfreeze()
    learn.bn_freeze(True)

    learn.fit(lrs/3, 2, wds=wd,cycle_len=16,use_clr=(20,8))
    
    learn.fit(lrs/8, 1, wds=wd,cycle_len=9,use_clr=(20,8))

    learn.save(f'512_SeResNext50_ph1_{k}')

# Experimenting with gradient clipping

In [205]:
def get_resnext50_model(md):
    learn = ResNextConv.pretrained(se_resnext50_32x4d, md, ps=0.5) #dropout 50%
    learn.opt_fn = optim.Adam
    learn.crit = FocalLoss()
    learn.metrics = [acc, f1_metric]
    learn.clip = 0.1 
    return learn 

In [206]:
def get_fold_model(sz, bs, k):
    md = get_fold(sz,bs,k)
    learn = get_resnext50_model(md)
    return learn

In [207]:
lr = 3e-3
wd = 1e-7
lrs=np.array([lr/10,lr/3,lr])

In [210]:
sz = 128
bs = 24

In [211]:
for k in range(folds):
    learn = get_fold_model(sz, bs, k)
    learn.unfreeze()
    learn.fit(lrs, 1, wds=wd, cycle_len=10, use_clr_beta=(10,10,0.85,0.9), use_wd_sched=True)
    
    pa = f'SEResNextFold_{sz}_{k}_gc0.12'
    learn.save(pa)

/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


epoch      trn_loss   val_loss   acc        f1_metric         
    0      1.359387   58.925576  0.924173   0.458162  
    1      1.427781   11266.822322 0.856858   1.045355        
  6%|▌         | 93/1542 [00:18<04:41,  5.15it/s, loss=1.42]

KeyboardInterrupt: 

In [ ]:
sz = 256
bs = 16

In [ ]:
for k in range(folds):
    learn = get_fold_model(sz, bs, k)
    learn.load(f'SEResNextFold_128_{k}_gc0.12')
    learn.unfreeze()
    
    learn.fit(lrs, 1, wds=wd, cycle_len=30, use_clr_beta=(20,8,0.85,0.9), use_wd_sched=True)
    
    pa = f'SEResNextFold_256_{k}_gc0.12'
    learn.save(pa)

In [ ]:
sz = 512
bs = 8

In [ ]:
for k in range(folds):
    learn = get_fold_model(sz, bs, k)
    learn.load(f'SEResNextFold_256_{k}_gc0.12')
    learn.unfreeze()
    learn.fit(lrs, 1, wds=wd, cycle_len=19, use_clr_beta=(20,8,0.85,0.9), use_wd_sched=True)
    
    pa = f'SEResNextFold_512_{k}_gc0.12'
    learn.save(pa)

In [ ]:
pred_array = np.zeros((11702, 28),dtype=float)

for k in range(folds):
    learn.load(f'SEResNextFold_512_{k}_gc0.12')
    preds_t,y_t = learn.TTA(n_aug=8, is_test=True)
    preds_t = np.stack(preds_t, axis=-1)
    preds_t = sigmoid_np(preds_t)
    pred_t = preds_t.max(axis=-1)
    pred_array = pred_t + pred_array

pred_array = pred_array / folds

In [ ]:
save_pred(pred_array,th_t,'protein_5folds_gc12.csv')

## Oversampling Edition

In [ ]:
def get_over_fold(sz, bs, k):
    aug_tfms = [RandomRotate(30, tfm_y=TfmType.NO),
                RandomDihedral(tfm_y=TfmType.NO),
                RandomLighting(0.05, 0.05, tfm_y=TfmType.NO)]
    
    stats = A([0.00505, 0.00331, 0.00344, 0.00519], [0.10038, 0.08131, 0.08284, 0.10179])
    tfms = tfms_from_stats(stats, sz, crop_type=CropType.NO, tfm_y=TfmType.NO, 
                aug_tfms=aug_tfms)
    
    trn_x = list(over_train_folds[k]['Id'])
    val_x = list(over_val_folds[k]['Id'])
    
    if len(trn_x)%bs == 0:
        ds = ImageData.get_ds(pdFilesDataset, (trn_x,TRAIN), 
                (val_x,TRAIN), tfms, test=(test_names,TEST))
    else:
        ds = ImageData.get_ds(pdFilesDataset, (trn_x[:-(len(trn_x)%bs)],TRAIN), 
                (val_x,TRAIN), tfms, test=(test_names,TEST))
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
    return md

def get_overfold_model(sz, bs, k):
    md = get_over_fold(sz,bs,k)
    learn = get_resnext50_model(md)
    learn.freeze_to(1)
    return learn

# 1 Cycle

### Find fold on 256

In [238]:
lr = 3e-3
wd = 1e-7
lrs=np.array([lr/10,lr/3,lr])

sz = 256
bs = 16

In [239]:
for k in range(folds):
    md = get_fold(sz,bs,k)
    learn = get_resnext50_model(md)
    
    learn.unfreeze()
    learn.fit(lrs, 1, wds=wd, cycle_len=9, use_clr_beta=(10,10,0.85,0.9), use_wd_sched=True)
    
    pa = f'SEResNextFold_{sz}_{k}_1cycle'
    learn.save(pa)

/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


epoch      trn_loss   val_loss   acc        f1_metric         
    0      1.328593   1.272191   0.947765   1.413501  
    1      1.137745   1.023102   0.953469   1.405513          
    2      1.043368   0.915269   0.958118   1.378022           
    3      1.026085   0.913677   0.958258   1.342311           
    4      0.936701   0.878641   0.959737   1.312736           
    5      0.887632   0.79616    0.962896   1.300051           
    6      0.833138   0.764952   0.963968   1.287958           
    7      0.811208   0.743725   0.965162   1.277324           
    8      0.792274   0.738354   0.965039   1.274591           


/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


epoch      trn_loss   val_loss   acc        f1_metric         
    0      1.366687   1.322389   0.947173   1.403577  
    1      1.121734   1.016569   0.953633   1.372699          
    2      1.036949   0.918403   0.957367   1.365476          
    3      0.974304   0.850032   0.960168   1.354051           
    4      0.975429   0.797373   0.962619   1.327153           
    5      0.891206   0.776034   0.96365    1.306319           
    6      0.87218    0.732663   0.965202   1.318009           
    7      0.854386   0.709041   0.966113   1.282925           
    8      0.798978   0.69504    0.96668    1.284465           


/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


epoch      trn_loss   val_loss   acc        f1_metric         
    0      1.320795   1.318231   0.94644    1.40908   
    1      1.096629   1.049087   0.953805   1.348498          
    2      1.02817    0.943915   0.956842   1.342956          
    3      0.993916   0.930248   0.957847   1.341957           
    4      0.902335   0.867526   0.959587   1.322497           
    5      0.919534   0.817346   0.962371   1.311022           
    6      0.833776   0.79827    0.963388   1.269858           
    7      0.81283    0.758793   0.964842   1.269543           
    8      0.790321   0.748633   0.965229   1.267038           


/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


epoch      trn_loss   val_loss   acc        f1_metric         
    0      1.349492   1.380949   0.944162   1.383587  
    1      1.13033    0.991769   0.954015   1.392682          
    2      1.005106   0.874963   0.958865   1.366259          
    3      1.013256   0.844407   0.960629   1.335342           
    4      0.960734   0.797777   0.962704   1.341817           
    5      0.886598   0.751285   0.96478    1.298729           
    6      0.879901   0.737562   0.964874   1.2824             
    7      0.811503   0.692786   0.967278   1.291397           
    8      0.80585    0.703499   0.966596   1.271528           


/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
/home/eigenstir/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


epoch      trn_loss   val_loss   acc        f1_metric         
    0      1.348071   1.363003   0.949994   1.32488   
    1      1.132455   0.967116   0.955429   1.395789          
    2      1.016522   0.870074   0.960179   1.33425           
    3      1.026964   0.846754   0.960929   1.366453           
    4      0.931521   0.807102   0.962607   1.330315           
    5      0.92518    0.749328   0.964548   1.320289           
    6      0.852273   0.732965   0.965661   1.281307           
    7      0.841811   0.790707   0.967464   1.312267           
    8      0.799584   0.779458   0.967649   1.286633           


In [ ]:
sz = 512
bs = 8

In [ ]:
for k in range(folds):
    md = get_fold(sz,bs,k)
    learn = get_resnext50_model(md)
    
    learn.load(f'SEResNextFold_256_{k}_1cycle')
    learn.unfreeze()
    learn.fit(lrs, 1, wds=wd, cycle_len=9, use_clr_beta=(10,10,0.85,0.9), use_wd_sched=True)
    pa = f'SEResNextFold_{sz}_{k}_1cycle'
    
    learn.save(pa)

### Full Loop

In [ ]:
for k in range(folds):
    # 128x128
    learn = get_fold_model(128, 24, k)
    lead.load(f'SEResNextFold_128_{k}')
    
    learn.fit(lr, 1, wds=wd, cycle_len=8, use_clr_beta=(5,4,0.85,0.9), use_wd_sched=True)
    
    learn.unfreeze()
    learn.bn_freeze(True)
    
    learn.fit(lrs, 1, wds=wd, cycle_len=10, use_clr_beta=(5,4,0.85,0.9), use_wd_sched=True)
    
    learn.save(f'SEResNextFold_256_{k}')
    
    #256x256
    learn = get_fold_model(256, 16, k)
    learn.load(f'SEResNextFold_256_{k}')
    
    learn.fit(lr, 1, wds=wd, cycle_len=5, use_clr_beta=(5,5,0.85,0.9), use_wd_sched=True)

    learn.unfreeze()
    learn.bn_freeze(True)
    
    learn.fit(lrs, 1, wds=wd, cycle_len=20, use_clr_beta=(20,8,0.85,0.9), use_wd_sched=True)

    learn.save(f'SeResNext50_256_{k}')
    
    #512 x 512 phase
    learn = get_fold_model(512, 8, k)
    learn.load(f'SeResNext50_256_{k}')

    learn.fit(lr,1, wds=wd, cycle_len=2,use_clr=(5,4))

    learn.unfreeze()
    learn.bn_freeze(True)

    learn.fit(lrs, 1, wds=wd, cycle_len=30, use_clr_beta=(20,8,0.85,0.9), use_wd_sched=True)

    learn.save(f'SeResNext50_512_{k}')

# Submission Functions

In [204]:
th_t = np.array([0.565,0.39,0.55,0.345,0.33,0.39,0.33,0.45,0.38,0.39,
               0.34,0.42,0.31,0.38,0.49,0.50,0.38,0.43,0.46,0.40,
               0.39,0.505,0.37,0.47,0.41,0.545,0.32,0.1])

def make_prediction():
    preds,y = learn.TTA(n_aug=8, is_test=True)
    preds = np.stack(preds, axis=-1)
    preds = sigmoid_np(preds)
    pred = preds.max(axis=-1)
    return pred

def save_pred(pred, th=0.5, fname='protein_classification.csv'):
    pred_list = []
    for line in pred:
        s = ' '.join(list([str(i) for i in np.nonzero(line>th)[0]]))
        pred_list.append(s)
        
    sample_df = pd.read_csv(SAMPLE)
    sample_list = list(sample_df.Id)
    pred_dic = dict((key, value) for (key, value) 
                in zip(learn.data.test_ds.fnames,pred_list))
    pred_list_cor = [pred_dic[id] for id in sample_list]
    df = pd.DataFrame({'Id':sample_list,'Predicted':pred_list_cor})
    df.to_csv(fname, header=True, index=False)

def sigmoid_np(x):
    return 1.0/(1.0 + np.exp(-x))